In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import polars as pl
import requests_cache

from moonbox import now, get_oneday, parse_oneday

In [ ]:
session = requests_cache.CachedSession("cache")

In [ ]:
def draw_moon(axes, f, direction, light="white", dark="black", eps=1.25e-2):
    """Draw a moon in arbitrary phase

    Note that, mathematically:
    - total area: pi*R^2
    - foreground: 0.5*pi*R^2 + 0.5*pi*R*r = 0.5*pi*R^2*(1+r/R)
    - f = 0.5*(1+r/R)
    - R*(2*f-1)=r

    Args:
        axes (matplotlib.Axes): plot axes
        f (float): fractional illumination
        direction (str): one of "waxing", "first", "waning", or "third"
        light (str): color of light pars of moon
        dark (str): color of dark parts of moon
        eps (float): fractional size reduction of "bottom" circle
    """
    center = (0, 0)
    radius = 0.9

    axes.set(aspect=1, xlim=(-1.0, 1.0), ylim=(-1.0, 1.0))
    axes.set_axis_off()

    if f == 0.0:
        back_color = dark
        half_side = None
        half_color = None
        ellipse_color = None
    elif direction == "waxing" and 0.0 < f < 0.5:
        back_color = light
        half_side = "left"
        half_color = dark
        ellipse_color = dark
    elif direction in ["waxing", "first"] and f == 0.5:
        back_color = light
        half_side = "left"
        half_color = dark
        ellipse_color = None
    elif direction == "waxing" and 0.5 < f < 1.0:
        back_color = dark
        half_side = "right"
        half_color = light
        ellipse_color = light
    elif f == 1.0:
        back_color = light
        half_side = None
        half_color = None
        ellipse_color = None
    elif direction == "waning" and 0.5 < f < 1.0:
        back_color = dark
        half_side = "left"
        half_color = light
        ellipse_color = light
    elif direction in ["waning", "third"] and f == 0.5:
        back_color = dark
        half_side = "left"
        half_color = light
        ellipse_color = None
    elif direction == "waning" and 0.0 < f < 0.5:
        back_color = light
        half_side = "right"
        half_color = dark
        ellipse_color = dark
    else:
        raise RuntimeError(f"bad values: f={f} direction={direction}")

    back = mpatches.Circle(center, radius * (1.0 - eps), ec="none")
    back.set(color=back_color)
    axes.add_artist(back)

    if half_side == "left":
        half = mpatches.Wedge(center, radius, 90, 270, ec="none")
        half.set(color=half_color)
        axes.add_artist(half)
    elif half_side == "right":
        half = mpatches.Wedge(center, radius, 270, 90, ec="none")
        half.set(color=half_color)
        axes.add_artist(half)
    elif half_side is None:
        pass
    else:
        raise ValueError(f"bad half: {half}")

    if ellipse_color is not None:
        artist = mpatches.Ellipse(center, 2 * radius * (2 * f - 1), 2 * radius)
        artist.set(color=ellipse_color)
        axes.add_artist(artist)

    return None

In [ ]:
year = now().year

dates = (
    pl.select(pl.date_range(pl.date(year, 1, 1), pl.date(year, 12, 31)))
    .to_series()
    .to_list()
)

data = [
    {"date": date} | parse_oneday(get_oneday(date.isoformat(), session=session))
    for date in dates
]

In [ ]:
def chunk(lst, pred, min_len=2):
    out = []
    chunk = None
    for elt in lst:
        if chunk is None:
            if pred(elt):
                chunk = [elt]
            else:
                pass
        elif pred(elt) and len(chunk) + 1 > min_len:
            out.append(chunk)
            chunk = [elt]
        else:
            chunk.append(elt)

    return out

In [ ]:
# group into lunar months
lunar_months = chunk(data, lambda x: x["phase"] == "New Moon")

n_months = len(lunar_months)
longest_month = max([len(x) for x in lunar_months])

scale = 1.0
fig, axs = plt.subplots(
    n_months,
    longest_month,
    figsize=(scale * longest_month, scale * n_months),
    layout="compressed",
)

# set everything to not visible, to deal with ragged rows
for row in range(n_months):
    for col in range(longest_month):
        axs[row, col].set_visible(False)

solar_month = None

for row, month in enumerate(lunar_months):
    for col, datum in enumerate(month):
        if datum["date"].month != solar_month:
            label = datum["date"].strftime("%b %-d")
            solar_month = datum["date"].month
        else:
            label = datum["date"].strftime("%-d")

        f = datum["illumination"] / 100
        direction = datum["phase"].split()[0].lower()

        ax = axs[row, col]
        ax.set_visible(True)
        draw_moon(ax, f=f, direction=direction, dark="0.2")
        ax.text(
            0.875,  # x-position of end of text
            1.0,  # y-position of top of text
            label,
            horizontalalignment="right",
            verticalalignment="bottom",
            transform=ax.transAxes,
            color="white",
        )

fig.suptitle(f"{year} Lunar Calendar", fontsize=60, color="white")
fig.set_facecolor("black")

plt.savefig("../output/lunar_calendar.pdf")
plt.savefig("../output/lunar_calendar.png")

plt.show()